# Nested Leave-One-Issue-Out Cross Validation For Random Forest On Contextual Features

In [4]:
import numpy as np
import pandas as pd
import xlrd as xl
from pandas import ExcelWriter
from pandas import ExcelFile
import pprint
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
import re
import pickle
from operator import itemgetter
import time, datetime
from functools import partial, update_wrapper
from openpyxl import load_workbook

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN 

from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as Imb_Pipeline

from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
# from imblearn.ensemble import BalancedBaggingClassifier

from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate, LeaveOneGroupOut
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score, make_scorer, confusion_matrix

pp = pprint.PrettyPrinter(indent=4)

## Ignore warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
np.warnings.filterwarnings('ignore')

#### Read from the pickled file:

In [2]:
all_data = pd.read_pickle('../../data/text_conv_data.pkl')

print("Size of corpus: "+str(len(all_data)))

Size of corpus: 4330


In [3]:
X = all_data.drop(['Code','Text Content','Document'], axis=1)
y = all_data['Code'].values
doc_groups = all_data['Document'].values

print("Number of unique labels: "+str(len(set(y))))

labels = list(set(y))
labels.sort()

pp.pprint(labels)

Number of unique labels: 13
[   'Action on Issue',
    'Bug Reproduction',
    'Contribution and Commitment',
    'Expected Behaviour',
    'Investigation and Exploration',
    'Social Conversation',
    'Motivation',
    'Observed Bug Behaviour',
    'Potential New Issues and Requests',
    'Solution Discussion',
    'Task Progress',
    'Solution Usage',
    'Workarounds']


## Nested Cross-Validation on Random Forest:

In [5]:
# To be used within GridSearch
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# To be used in outer CV
logo = LeaveOneGroupOut()
outer_cv = logo.split(X,y,groups=doc_groups)

### Define Pipelines:

1. Pipeline1: class_weight=balanced
2. Pipeline2: SMOTE

In [5]:
pipeline1 = Pipeline([
    ('clf', RandomForestClassifier(class_weight='balanced')),
])

pipeline2 = Imb_Pipeline([
    ('smote', SMOTE()),
    ('clf', RandomForestClassifier())
])

### Hyperparameters to search
parameters = {
    'clf__n_estimators': (10, 50, 100),
    'clf__min_samples_split': (2, 5, 10),
}

## Nested Cross Validation Using GridSearch

In [6]:
### Define and create the scoring functions

def score_func(y_true, y_pred, score_index, i):
    return(precision_recall_fscore_support(y_true,y_pred,labels=labels)[score_index][i])

def avg_score(y_true, y_pred, score_index):
    return precision_recall_fscore_support(y_true,y_pred,average='weighted')[score_index]

def sum_support(y_true, y_pred):
    return len(y_true)

### Create partials for each of the metrics returned
score_funcs = {v: partial(score_func, score_index=k) for k, v in {0:'precision',1:'recall',2:'fscore',3:'support'}.items()}
prec_score = partial(score_func, score_index=0)
update_wrapper(prec_score,score_func)
rec_score = partial(score_func, score_index=1)
update_wrapper(rec_score,score_func)
f_score = partial(score_func, score_index=2)
update_wrapper(f_score,score_func)
support_score = partial(score_func, score_index=3)
update_wrapper(support_score,score_func)

### Create a callable scoring function for each of the metrics for each classification label
scorer = {}
for label_id in range(0,13):
    scorer['label'+str(label_id)+'_precision'] = make_scorer(prec_score, i=label_id)
    scorer['label'+str(label_id)+'_recall'] = make_scorer(rec_score, i=label_id)
    scorer['label'+str(label_id)+'_fscore'] = make_scorer(f_score, i=label_id)
    scorer['label'+str(label_id)+'_support'] = make_scorer(support_score, i=label_id)

### Create a callable scoring function for avg/total of the metrics across classification labels
scorer['avg_precision'] = make_scorer(avg_score,score_index=0)
scorer['avg_recall'] = make_scorer(avg_score,score_index=1)
scorer['avg_fscore'] = make_scorer(avg_score,score_index=2)
scorer['total_support'] = make_scorer(sum_support)

### Perform Nested cross-validation on Pipeline1
outer_cv = logo.split(X,y,groups=doc_groups)

start = time.time()
clf1 = GridSearchCV(pipeline1, parameters, cv=inner_cv, scoring='f1_weighted')
clf1_results = cross_validate(clf1, X=X, y=y, cv=outer_cv, scoring=scorer)
print("Completed Pipeline1 scenario in "+ str(datetime.timedelta(seconds=(time.time()-start))))

### Perform Nested cross-validation on Pipeline2
outer_cv = logo.split(X,y,groups=doc_groups)

start = time.time()
clf2 = GridSearchCV(pipeline2, parameters, cv=inner_cv, scoring='f1_weighted')
clf2_results = cross_validate(clf2, X=X, y=y, cv=outer_cv, scoring=scorer)
print("Completed Pipeline2 scenario in "+ str(datetime.timedelta(seconds=(time.time()-start))))

Completed Pipeline1 scenario in 0:31:10.909444
Completed Pipeline2 scenario in 1:10:38.189200


# Display and Save Training and Testing Results for each Fold:

## Pipeline1 Results:

In [13]:
train_report = pd.DataFrame(columns=['Precision', 'Recall', 'F1-score', 'Support'])
test_report = pd.DataFrame(columns=['Precision', 'Recall', 'F1-score', 'Support'])

result_dict = {}

# writer = pd.ExcelWriter('../../results/LOIO.xlsx')
book = load_workbook('../../results/LOIO.xlsx')
writer = pd.ExcelWriter('../../results/LOIO.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

datalength = 0

for i in range(0,15):
    for label_id in range(0,13):
        train_report.loc[labels[label_id], :] = [clf1_results['train_label'+str(label_id)+'_precision'][i],clf1_results['train_label'+str(label_id)+'_recall'][i],clf1_results['train_label'+str(label_id)+'_fscore'][i],clf1_results['train_label'+str(label_id)+'_support'][i]]
        test_report.loc[labels[label_id], :] = [clf1_results['test_label'+str(label_id)+'_precision'][i],clf1_results['test_label'+str(label_id)+'_recall'][i],clf1_results['test_label'+str(label_id)+'_fscore'][i],clf1_results['test_label'+str(label_id)+'_support'][i]]

    train_report.loc['Avg/Total', :] = [clf1_results['train_avg_precision'][i],clf1_results['train_avg_recall'][i],clf1_results['train_avg_fscore'][i],clf1_results['train_total_support'][i]]
    test_report.loc['Avg/Total', :] = [clf1_results['test_avg_precision'][i],clf1_results['test_avg_recall'][i],clf1_results['test_avg_fscore'][i],clf1_results['test_total_support'][i]]

    fold_index = pd.DataFrame(data=[{'Fold':'Fold '+str(i)}])
    fold_index.to_excel(writer,'RCC',startrow=datalength, index=False)
    datalength+=(len(fold_index)+2)
    train_report.to_excel(writer,'RCC',startrow=datalength)
    datalength+=(len(train_report)+2)
    test_report.to_excel(writer,'RCC',startrow=datalength)
    datalength+=(len(test_report)+2)

    result_dict['RCC_train_'+str(i)] = train_report
    result_dict['RCC_test_'+str(i)] = test_report

    train_report = train_report.astype(float).round(2)
    test_report = test_report.astype(float).round(2)

    print("\n------------------------- FOLD "+str(i)+": -------------------------")
    print("\nTraining Results:")
    print(train_report)
    print("\nTest Results:")
    print(test_report)

writer.save()


------------------------- FOLD 0: -------------------------

Training Results:
                                   Precision  Recall  F1-score  Support
Action on Issue                         0.87    1.00      0.93      53.0
Bug Reproduction                        0.97    0.99      0.98     245.0
Contribution and Commitment             0.94    1.00      0.97      58.0
Expected Behaviour                      0.96    0.98      0.97     119.0
Investigation and Exploration           0.96    0.98      0.97     377.0
Social Conversation                     0.96    0.93      0.94     546.0
Motivation                              0.95    0.98      0.97     261.0
Observed Bug Behaviour                  0.90    1.00      0.95     131.0
Potential New Issues and Requests       0.98    1.00      0.99     208.0
Solution Discussion                     1.00    0.93      0.97    1338.0
Task Progress                           0.95    1.00      0.98     104.0
Solution Usage                          0.92 

## Pipeline2 Results:

In [12]:
train_report = pd.DataFrame(columns=['Precision', 'Recall', 'F1-score', 'Support'])
test_report = pd.DataFrame(columns=['Precision', 'Recall', 'F1-score', 'Support'])

result_dict = {}

# writer = pd.ExcelWriter('../../results/LOIO.xlsx')
book = load_workbook('../../results/LOIO.xlsx')
writer = pd.ExcelWriter('../../results/LOIO.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

datalength = 0

for i in range(0,15):
    for label_id in range(0,13):
        train_report.loc[labels[label_id], :] = [clf2_results['train_label'+str(label_id)+'_precision'][i],clf2_results['train_label'+str(label_id)+'_recall'][i],clf2_results['train_label'+str(label_id)+'_fscore'][i],clf2_results['train_label'+str(label_id)+'_support'][i]]
        test_report.loc[labels[label_id], :] = [clf2_results['test_label'+str(label_id)+'_precision'][i],clf2_results['test_label'+str(label_id)+'_recall'][i],clf2_results['test_label'+str(label_id)+'_fscore'][i],clf2_results['test_label'+str(label_id)+'_support'][i]]

    train_report.loc['Avg/Total', :] = [clf2_results['train_avg_precision'][i],clf2_results['train_avg_recall'][i],clf2_results['train_avg_fscore'][i],clf2_results['train_total_support'][i]]
    test_report.loc['Avg/Total', :] = [clf2_results['test_avg_precision'][i],clf2_results['test_avg_recall'][i],clf2_results['test_avg_fscore'][i],clf2_results['test_total_support'][i]]

    fold_index = pd.DataFrame(data=[{'Fold':'Fold '+str(i)}])
    fold_index.to_excel(writer,'RCS',startrow=datalength, index=False)
    datalength+=(len(fold_index)+2)
    train_report.to_excel(writer,'RCS',startrow=datalength)
    datalength+=(len(train_report)+2)
    test_report.to_excel(writer,'RCS',startrow=datalength)
    datalength+=(len(test_report)+2)

    result_dict['RCS_train_'+str(i)] = train_report
    result_dict['RCS_test_'+str(i)] = test_report

    train_report = train_report.astype(float).round(2)
    test_report = test_report.astype(float).round(2)

    print("\n------------------------- FOLD "+str(i)+": -------------------------")
    print("\nTraining Results:")
    print(train_report)
    print("\nTest Results:")
    print(test_report)

writer.save()


------------------------- FOLD 0: -------------------------

Training Results:
                                   Precision  Recall  F1-score  Support
Action on Issue                          1.0     1.0       1.0      53.0
Bug Reproduction                         1.0     1.0       1.0     245.0
Contribution and Commitment              1.0     1.0       1.0      58.0
Expected Behaviour                       1.0     1.0       1.0     119.0
Investigation and Exploration            1.0     1.0       1.0     377.0
Social Conversation                      1.0     1.0       1.0     546.0
Motivation                               1.0     1.0       1.0     261.0
Observed Bug Behaviour                   1.0     1.0       1.0     131.0
Potential New Issues and Requests        1.0     1.0       1.0     208.0
Solution Discussion                      1.0     1.0       1.0    1338.0
Task Progress                            1.0     1.0       1.0     104.0
Solution Usage                           1.0 

### Get lists of F1 scores and support-weighted average F1 score

In [14]:
avg_f1 = []
for i in range(0,15):
    avg_f1.append(float('%.4f' % round(clf1_results['test_avg_fscore'][i],4)))
    
print('Pipeline1:')
print(avg_f1)

avg_f1 = []
for i in range(0,15):
    avg_f1.append(float('%.4f' % round(clf2_results['test_avg_fscore'][i],4)))

print('\nPipeline2:')
print(avg_f1)

Pipeline1:
[0.4687, 0.2359, 0.1745, 0.2783, 0.4808, 0.1661, 0.3022, 0.2125, 0.189, 0.3157, 0.1964, 0.5451, 0.0906, 0.3452, 0.4987]

Pipeline2:
[0.4886, 0.2043, 0.2256, 0.249, 0.3293, 0.2033, 0.2022, 0.1882, 0.236, 0.2321, 0.2049, 0.4612, 0.1229, 0.3217, 0.4392]


In [15]:
sum = 0
den = 0
for i in range(0,15):
    sum += clf2_results['test_avg_fscore'][i] * clf2_results['test_total_support'][i]
    den += clf2_results['test_total_support'][i]
print('Pipeline2: Total weighted F-Score: '+('%.4f' % round(sum/den,4)))

sum = 0
den = 0
for i in range(0,15):
    sum += clf3_results['test_avg_fscore'][i] * clf3_results['test_total_support'][i]
    den += clf3_results['test_total_support'][i]
print('Pipeline3: Total weighted F-Score: '+('%.4f' % round(sum/den,4)))

Pipeline1: Total weighted F-Score: 0.2948
Pipeline2: Total weighted F-Score: 0.2680
